In [1]:
import numpy as np
import pandas as pd
import torch.nn as nn
import torch
import os
import random

from functions.parse_data import synth_dataloader
from multivariate_quantile_regression.network_model import QuantileNetwork

from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
from sklearn.model_selection import train_test_split

/tmp/ipykernel_9488/3516985000.py:2: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd
2024-03-15 11:42:34.687642: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-03-15 11:42:34.721333: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-03-15 11:42:

In [2]:
# Check if CUDA (GPU support) is available
if torch.cuda.is_available():
    # CUDA is available, so let's set default device to GPU
    torch.set_default_device(0)
    print("CUDA is available. Using GPU.")
else:
    # CUDA is not available, so let's use the CPU
    print("CUDA is not available. Using CPU.")

# Example usage:
tensor = torch.randn(3, 3)  # Create a tensor on the selected device
print("Tensor is on device:", tensor.device)

CUDA is available. Using GPU.
Tensor is on device: cuda:0


In [3]:
#Load data and inspect
df = synth_dataloader('SMHIdata')
df.head(10)

,Cloud_B02,Cloud_B03,Cloud_B04,Cloud_B05,Cloud_B06,Cloud_B07,Cloud_B08,Cloud_B08A,Cloud_B09,Cloud_B10,...,Clear_B11,Clear_B12,Sat_Zenith_Angle,Sun_Zenith_Angle,Azimuth_Diff_Angle,COT,Cloud_Type,Profile_ID,GOT,Water_Vapor
0,0.94195,0.87799,0.92936,0.93407,0.95181,0.96217,0.92871,0.97181,0.49957,0.04136,...,0.12946,0.18888,4.53,52.05,167.66,5.897,3,3335,0.126,0.35
1,0.30422,0.40100,0.27834,0.57800,1.01964,1.02787,1.00519,1.03599,0.59139,0.01055,...,0.71532,0.36823,12.85,41.68,161.91,1.275,2,1996,0.126,0.31
2,0.28715,0.25066,0.30366,0.29214,0.34088,0.40079,0.37376,0.48750,0.02092,0.00067,...,0.86232,0.63915,14.53,79.23,168.52,1.799,1,6796,0.127,4.04
3,0.27146,0.33719,0.19841,0.46411,0.88787,0.89584,0.87746,0.90439,0.51811,0.00561,...,0.56307,0.23663,6.54,70.23,165.49,0.519,2,3701,0.123,0.22
4,0.39689,0.38594,0.32623,0.37338,0.60678,0.66895,0.55343,0.70168,0.01513,0.00049,...,0.56472,0.20853,8.56,75.15,148.48,8.569,2,6345,0.128,5.40
5,0.75592,0.65853,0.70670,0.71369,0.73147,0.74182,0.71190,0.75003,0.32287,0.00328,...,0.92187,0.87515,12.38,73.72,153.05,16.874,3,1419,0.126,0.51
6,0.34660,0.30719,0.27459,0.35268,0.72965,0.74705,0.69842,0.75893,0.18345,0.00089,...,0.64499,0.27212,14.74,73.05,13.63,3.589,3,424,0.122,0.99
7,0.22494,0.33843,0.19615,0.53821,0.98347,0.99256,0.96757,1.00132,0.53669,0.00511,...,0.61388,0.28605,8.24,53.20,146.76,0.624,2,3427,0.125,0.35
8,0.57982,0.62369,0.57909,0.75147,0.99608,1.04055,0.94017,1.07212,0.43014,0.03328,...,0.94942,0.49475,11.57,38.00,117.48,13.909,1,6884,0.122,1.88
9,0.83664,0.80620,0.81432,0.78581,0.81063,0.84108,0.78542,0.85153,0.26374,0.00122,...,0.90206,0.89850,9.69,45.83,23.04,24.937,3,7455,0.106,1.77


In [4]:
#Set columns for X and y (input/output features)
X_cols = ['Cloud_B02','Cloud_B03','Cloud_B04','Cloud_B05','Cloud_B06',
          'Cloud_B07','Cloud_B08','Cloud_B08A','Cloud_B09','Cloud_B10','Cloud_B11','Cloud_B12',
          'Sat_Zenith_Angle','Sun_Zenith_Angle','Azimuth_Diff_Angle']
y_cols = ['Clear_B02','Clear_B03','Clear_B04','Clear_B05','Clear_B06',
          'Clear_B07','Clear_B08','Clear_B08A','Clear_B09','Clear_B10','Clear_B11','Clear_B12']

#Find X and y
X=df[X_cols]
y=df[y_cols]

#Separate testdata from rest for 80/10/10 Train/Val/Test split
X_trainval, X_test, y_trainval, y_test=train_test_split(X,y,test_size=0.1,random_state=313)

#Add noise to X_test, 0 mean with stdev equal to 3% of mean of each feature
np.random.seed(313)
X_test = X_test + np.random.randn(np.shape(X_test)[0],np.shape(X_test)[1]) * np.mean(X.to_numpy(),axis=0)*0.03

#Choose band 11 for evaluation (idx=10)
band_idx = 10

In [5]:
#Set up which quantiles to estimate, and find index of estimator (q=0.5)
quantiles=np.array([0.1,0.5,0.9])
est= np.where(quantiles==0.5)[0].item()

#Set up algorithm parameters for both cases
val_size=0.1
num_models=5 #Set number of models in ensemble
batch_size=500
nepochs=1000
lr=0.003
noise_ratio = 0.03
early_break=True

Case 1: Multiple-Output. Predict all frequency bands but only evaluate performance on B11

In [6]:
#Choose if to save models and metrics, if so set path
save = True
if save:
    test_name = "Mult_v_Sing_Out-Mult"
    main_filepath = 'pytorch_models/'+test_name

#Set up NN structure
no_nodes = 100

sequence= lambda: nn.Sequential(
    nn.Linear(len(X_cols),no_nodes),
    nn.ReLU(),
    nn.Linear(no_nodes,no_nodes),
    nn.ReLU(),
    nn.Linear(no_nodes,no_nodes),
    nn.ReLU(),
    nn.Linear(no_nodes,no_nodes),
    nn.ReLU(),
    nn.Linear(no_nodes,no_nodes),
    nn.ReLU(),
    nn.Linear(no_nodes,no_nodes),
    nn.ReLU(),
    nn.Linear(no_nodes, len(quantiles)*len(y_cols)) #Output dimesion is number of quantiles times number of target variables
)

#Initalize models
models = [QuantileNetwork(quantiles=quantiles) for _ in range(num_models)]

#Train models
for i,model in enumerate(models):
    #Find new train/val splits for each model for robustness
    validation_indices=np.array(random.sample(range(len(X_trainval['Cloud_B02'])), int(len(X['Cloud_B02'])*val_size)))
    train_indices=[i for i in range(len(X_trainval['Cloud_B02'])) if np.any(validation_indices==i)==False]  
    #Fit model
    model.fit(X_trainval.to_numpy(),y_trainval.to_numpy(), 
            train_indices=train_indices, 
            validation_indices=validation_indices, 
            batch_size=batch_size,
            nepochs=nepochs,
            sequence=sequence(),
            lr=lr,
            noise_ratio=noise_ratio,
            early_break=early_break)
    
    #Save models if wanted
    if save:
        filepath=main_filepath+'/model'+str(i)
        os.makedirs(filepath,exist_ok=True)
        torch.save(model,filepath+'/model_file')
    

Epoch 451


Batch number: 100%|██████████| 320/320 [00:00<00:00, 326.35it/s]

Training loss [2.0806575] Validation loss [2.12452]
Epoch 452



Batch number: 100%|██████████| 320/320 [00:01<00:00, 308.81it/s]

Training loss [2.0757248] Validation loss [2.1101832]
Epoch 453


...

Training loss [2.0723538] Validation loss [2.120971]
Epoch 458


Batch number: 100%|██████████| 320/320 [00:00<00:00, 355.72it/s]


Training loss [2.0829942] Validation loss [2.1323202]
Epoch 459


Batch number: 100%|██████████| 320/320 [00:00<00:00, 331.56it/s]

Training loss [2.0773761] Validation loss [2.1172578]
---No improvement in 100 epochs, broke early---
Best model out of total max epochs found at epoch 359
With validation loss: 2.1021151542663574


In [7]:
#Initialize dataframe for error metrics and array for ensemble predictions
mult_model_metrics=pd.DataFrame(columns=['Ensemble_mean','Ensemble_index','MSE','PSNR','R2_score','Mean_Quantile_Loss'])
preds_total=[]
#Make predictions and evaluate
for i,model in enumerate(models):
    preds = model.predict(X_test.to_numpy())
    #Keep track of ensemble prediction
    if i==0:
        preds_total=preds
    else:
        preds_total=preds_total+preds

    #Find errors
    mse=mean_squared_error(y_test.to_numpy()[:,band_idx],preds[:,band_idx,est])
    psnr=QuantileNetwork.PSNR(y_test.to_numpy()[:,band_idx],preds[:,band_idx,est])
    r2=r2_score(y_test.to_numpy()[:,band_idx],preds[:,band_idx,est])
    mean_quantile=QuantileNetwork.mean_marginal_loss(y_test.to_numpy()[:,band_idx],preds[:,band_idx,:],quantiles)
    #Add to dataframe
    tmp_metrics=pd.DataFrame(data=[[False,i,mse,psnr,r2,mean_quantile]],columns=['Ensemble_mean','Ensemble_index','MSE','PSNR','R2_score','Mean_Quantile_Loss'])
    mult_model_metrics=pd.concat([mult_model_metrics,tmp_metrics])


#Now do the same for ensemble predictions
preds_total=preds_total/num_models

mse=mean_squared_error(y_test.to_numpy()[:,band_idx],preds_total[:,band_idx,est])
psnr=QuantileNetwork.PSNR(y_test.to_numpy()[:,band_idx],preds_total[:,band_idx,est])
r2=r2_score(y_test.to_numpy()[:,band_idx],preds_total[:,band_idx,est])
mean_quantile=QuantileNetwork.mean_marginal_loss(y_test.to_numpy()[:,band_idx],preds_total[:,band_idx,:],quantiles)

tmp_metrics=pd.DataFrame(data=[[True,np.nan,mse,psnr,r2,mean_quantile]],columns=['Ensemble_mean','Ensemble_index','MSE','PSNR','R2_score','Mean_Quantile_Loss'])
mult_model_metrics=pd.concat([mult_model_metrics,tmp_metrics])

#Save metrics if we want to
if save:
    mult_model_metrics=mult_model_metrics.reset_index(drop=True)
    mult_model_metrics.to_csv(main_filepath+'/model_metrics.csv',index=False)
    

/tmp/ipykernel_9488/3652957587.py:20: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  mult_model_metrics=pd.concat([mult_model_metrics,tmp_metrics])


Case 2: Single Output. Predict only band 11

In [8]:
#Choose if to save models and metrics, if so set path
save = True
if save:
    test_name = "Mult_v_Sing_Out-Sing"
    main_filepath = 'pytorch_models/'+test_name

#Set up NN structure
no_nodes = 100

sequence= lambda: nn.Sequential(
    nn.Linear(len(X_cols),no_nodes),
    nn.ReLU(),
    nn.Linear(no_nodes,no_nodes),
    nn.ReLU(),
    nn.Linear(no_nodes,no_nodes),
    nn.ReLU(),
    nn.Linear(no_nodes,no_nodes),
    nn.ReLU(),
    nn.Linear(no_nodes, len(quantiles)) #Output dimesion is number of quantiles times number of target variables (one)
)

#Initalize models
models = [QuantileNetwork(quantiles=quantiles) for _ in range(num_models)]

#Train models
for i,model in enumerate(models):
    #Find new train/val splits for each model for robustness
    validation_indices=np.array(random.sample(range(len(X_trainval['Cloud_B02'])), int(len(X['Cloud_B02'])*val_size)))
    train_indices=[i for i in range(len(X_trainval['Cloud_B02'])) if np.any(validation_indices==i)==False]  
    #Fit model with y being only band 11
    model.fit(X_trainval.to_numpy(),y_trainval.to_numpy()[:,band_idx], 
            train_indices=train_indices, 
            validation_indices=validation_indices, 
            batch_size=batch_size,
            nepochs=nepochs,
            sequence=sequence(),
            lr=lr,
            noise_ratio=noise_ratio,
            early_break=early_break)
    
    #Save models if wanted
    if save:
        filepath=main_filepath+'/model'+str(i)
        os.makedirs(filepath,exist_ok=True)
        torch.save(model,filepath+'/model_file')
    

Epoch 331


Batch number: 100%|██████████| 320/320 [00:00<00:00, 340.86it/s]

Training loss [0.3217034] Validation loss [0.33950654]
Epoch 332



Batch number: 100%|██████████| 320/320 [00:00<00:00, 339.21it/s]

Training loss [0.3224375] Validation loss [0.3387881]
Epoch 333



Batch number: 100%|██████████| 320/320 [00:00<00:00, 337.24it/s]

Training loss [0.32211912] Validation loss [0.33634546]
---No improvement in 100 epochs, broke early---
Best model out of total max epochs found at epoch 233
With validation loss: 0.3310999572277069


In [9]:
#Initialize dataframe for error metrics and array for ensemble predictions
sing_model_metrics=pd.DataFrame(columns=['Ensemble_mean','Ensemble_index','MSE','PSNR','R2_score','Mean_Quantile_Loss'])
preds_total=[]
#Make predictions and evaluate
for i,model in enumerate(models):
    preds = model.predict(X_test.to_numpy())
    #Keep track of ensemble prediction
    if i==0:
        preds_total=preds
    else:
        preds_total=preds_total+preds

    #Find errors
    mse=mean_squared_error(y_test.to_numpy()[:,band_idx],preds[:,est])
    psnr=QuantileNetwork.PSNR(y_test.to_numpy()[:,band_idx],preds[:,est])
    r2=r2_score(y_test.to_numpy()[:,band_idx],preds[:,est])
    mean_quantile=QuantileNetwork.mean_marginal_loss(y_test.to_numpy()[:,band_idx],preds[:,:],quantiles)
    #Add to dataframe
    tmp_metrics=pd.DataFrame(data=[[False,i,mse,psnr,r2,mean_quantile]],columns=['Ensemble_mean','Ensemble_index','MSE','PSNR','R2_score','Mean_Quantile_Loss'])
    sing_model_metrics=pd.concat([sing_model_metrics,tmp_metrics])


#Now do the same for ensemble predictions
preds_total=preds_total/num_models

mse=mean_squared_error(y_test.to_numpy()[:,band_idx],preds_total[:,est])
psnr=QuantileNetwork.PSNR(y_test.to_numpy()[:,band_idx],preds_total[:,est])
r2=r2_score(y_test.to_numpy()[:,band_idx],preds_total[:,est])
mean_quantile=QuantileNetwork.mean_marginal_loss(y_test.to_numpy()[:,band_idx],preds_total[:,:],quantiles)

tmp_metrics=pd.DataFrame(data=[[True,np.nan,mse,psnr,r2,mean_quantile]],columns=['Ensemble_mean','Ensemble_index','MSE','PSNR','R2_score','Mean_Quantile_Loss'])
sing_model_metrics=pd.concat([sing_model_metrics,tmp_metrics])

#Save metrics if we want to
if save:
    sing_model_metrics=sing_model_metrics.reset_index(drop=True)
    sing_model_metrics.to_csv(main_filepath+'/model_metrics.csv',index=False)

/tmp/ipykernel_9488/3998160532.py:20: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  sing_model_metrics=pd.concat([sing_model_metrics,tmp_metrics])


In [10]:
#Display mult results
mult_model_metrics

,Ensemble_mean,Ensemble_index,MSE,PSNR,R2_score,Mean_Quantile_Loss
0,False,0,0.022352,16.408836,0.557139,0.076938
1,False,1,0.022157,16.446792,0.560993,0.076701
2,False,2,0.022868,16.309744,0.546919,0.076960
3,False,3,0.022279,16.423038,0.558585,0.076712
4,False,4,0.022257,16.427350,0.559023,0.077108
5,True,NaN,0.021212,16.636054,0.579714,0.074210


In [11]:
#Display sing results
sing_model_metrics

,Ensemble_mean,Ensemble_index,MSE,PSNR,R2_score,Mean_Quantile_Loss
0,False,0,0.022992,16.286216,0.544457,0.075848
1,False,1,0.022951,16.294024,0.545276,0.075780
2,False,2,0.022900,16.303579,0.546275,0.076441
3,False,3,0.023017,16.281523,0.543965,0.075758
4,False,4,0.023103,16.265236,0.542251,0.076198
5,True,NaN,0.021672,16.542948,0.570606,0.073929
